In [ ]:
!pip install transformers datasets evaluate scikit-learn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("ai4bharat/indicqa", split="train")  # 'validation' or 'train'


In [ ]:
dataset[0]

{'version': 1,
 'data': {'paragraphs': [{'context': '"অহল্যা" নামটো দুটা ভাগত ভাগ কৰিব পাৰি: "অ" আৰু "হল্যা। "  "অ" এটা নঞৰ্থক উপসৰ্গ আৰু সংস্কৃত অভিধানসমূহত "হল্যা" শব্দটিৰ সংজ্ঞা এটা নাঙল (হাল), হালকৰ্ষণ অথবা অংগবৈকল্যৰ সৈতে যুক্ত কৰা হৈছে। ৰামায়ণৰ উত্তৰাকাণ্ডত ব্ৰহ্মাই সংস্কৃত "অহল্যা" নামটিৰ সংজ্ঞা আগবঢ়াই কৈছে যে "যি অসৌন্দৰ্যৰ অখ্যাতিৰ পৰা মুক্ত" অথবা "যি নিখুঁত সুন্দৰী। " এই প্ৰসংগত ইন্দ্ৰক তেওঁ কি দৰে অহল্যাৰ শৰীৰৰ সকলো অংশৰ মাজেৰে সকলো সৃষ্টিৰ বিশেষ সৌন্দৰ্য প্ৰকাশ কৰিছে, সেই বিষয়ে ব্যক্ত কৰিছে। কোনো কোনো সংস্কৃত অভিধানত যিহেতু অহল্যা শব্দটিৰ অনুবাদত "অকৰ্ষিত" অভিধাটো ব্যৱহাৰ কৰিছে,  সেই হেতু কোনো কোনো আধুনিক লেখকে এই শব্দটো যৌনসংগমৰ ইংগিত সূচক বুলি দাবী কৰে। তেওঁলোকৰ মতে, নামটি কোনো কুমাৰী অথবা মাতৃচৰিত্ৰৰ দ্যোতক। এই অৰ্থটো তেওঁৰ চৰিত্ৰৰ সৈতে খাপ খায়। কাৰণ, ভিন্ন ভিন্ন দৃষ্টিকোণৰ পৰা ইন্দ্ৰই ঢুকি পোৱাৰ পৰা যিকোনো প্ৰকাৰে নিলগত থকা বুলি ভবা অহল্যাৰ চৰিত্ৰ সন্দৰ্ভত এই কথা খাপ খাই পৰে।',
    'qas': [{'answers': [{'answer_start': None, 'text': ''}],
      'category': 'NO',
   

In [ ]:
flat_data = []

for entry in dataset['data']:
    for para in entry['paragraphs']:
        context = para['context']
        for qa in para['qas']:
            question = qa['question']
            answers = qa['answers']
            if not answers or len(answers[0]['text'].strip()) == 0:
                continue
            answer_text = answers[0]['text']
            flat_data.append({
                "context": context,
                "question": question,
                "answer": answer_text,
                "category": qa.get("category", "general"),
                "id": qa["id"]
            })


In [ ]:
from datasets import Dataset

flat_dataset = Dataset.from_list(flat_data)
flat_dataset = flat_dataset.shuffle(seed=42).select(range(50))  # limit for testing


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def predict_answer_qa(question, context):
    inputs = tokenizer(question, context, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)

    start_logits = outputs.start_logits
    end_logits = outputs.end_logits

    start_idx = torch.argmax(start_logits)
    end_idx = torch.argmax(end_logits)

    tokens = inputs["input_ids"][0][start_idx:end_idx + 1]
    return tokenizer.decode(tokens, skip_special_tokens=True)


In [ ]:
def normalize(text):
    return text.strip().lower()

def exact_match(pred, true):
    return int(normalize(pred) == normalize(true))

def compute_f1(pred, true):
    pred_tokens = normalize(pred).split()
    true_tokens = normalize(true).split()
    common = set(pred_tokens) & set(true_tokens)
    if not common:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(true_tokens)
    return 2 * precision * recall / (precision + recall)


In [ ]:
def normalize(text):
    return text.strip().lower()

def exact_match(pred, true):
    return int(normalize(pred) == normalize(true))

def compute_f1(pred, true):
    pred_tokens = normalize(pred).split()
    true_tokens = normalize(true).split()
    common = set(pred_tokens) & set(true_tokens)
    if not common:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(true_tokens)
    return 2 * precision * recall / (precision + recall)


In [ ]:
results = []

for sample in flat_dataset:
    question = sample["question"]
    context = sample["context"]
    true_answer = sample["answer"]
    pred = predict_answer_qa(question, context)

    em = exact_match(pred, true_answer)
    f1 = compute_f1(pred, true_answer)

    results.append({
        "id": sample["id"],
        "question": question,
        "true": true_answer,
        "pred": pred,
        "em": em,
        "f1": f1
    })


In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df[["question", "true", "pred", "em", "f1"]].head()


,question,true,pred,em,f1
0,ఏ సంవత్సరంలో కలకత్తాలో సమావేశమైన ఆలిండియా సంస్...,1924,,0,0.000000
1,आर्यभट्ट द्वारा लिखित तीन पुस्तकों का क्या नाम...,"दशगीतिका, आर्यभटीय और तंत्र",,0,0.000000
2,எந்த துறை எண் மற்றும் தரத்தில் வளர்ந்துள்ளது?,மருத்துவத் துறை,,0,0.000000
3,ਸ੍ਰੀ ਗੁਰੂ ਤੇਗ ਬਹਾਦਰ ਜੀ ਦਰਬਾਰ ਸਾਹਿਬ ਦੇ ਦਰਸ਼ਨਾਂ ...,1664,ਤੇਗ ਬਹਾਦਰ ਜੀ ਦਰਬਾਰ ਸਾਹਿਬ ਦੇ ਦਰਸ਼ਨਾਂ ਲਈ ਕਦੋ ਆਏ?...,0,0.012579
4,कोणत्या भाषेत वृत्तपत्रे प्रकाशित झाले?,मराठीतून,पु,0,0.000000


In [ ]:
print("Exact Match:", df["em"].mean())
print("F1 Score:", df["f1"].mean())


Exact Match: 0.0
F1 Score: 0.008242692332029117
